In [60]:
import yfinance as yf

def get_correct_ticker(user_input):
    """
    Maps user-friendly tickers to Yahoo Finance tickers for commodities, indices, and Forex.
    """
    ticker_map = {
        "XAUUSD": "GC=F",  # Gold futures
        "XAGUSD": "SI=F",  # Silver futures
        "BTCUSD": "BTC-USD",  # Bitcoin
        "ETHUSD": "ETH-USD",  # Ethereum
        "US30": "^DJI",  # Dow Jones Index
        "SPX500": "^GSPC",  # S&P 500 Index
        "NAS100": "^NDX",  # Nasdaq Index
        "FTSE100": "^FTSE",  # FTSE Index
        "DAX40": "^GDAXI",  # DAX Index
        "NIKKEI225": "^N225",  # Nikkei Index
        "HSI": "^HSI",  # Hang Seng Index
    }
    return ticker_map.get(user_input.upper(), user_input.upper() + "=X")

def get_instrument_price(ticker):
    """
    Fetch the current price of the instrument using Yahoo Finance.
    """
    try:
        data = yf.Ticker(ticker)
        price = data.history(period="1d")['Close'].iloc[-1]
        return price
    except Exception as e:
        print(f"Error fetching price for {ticker}: {e}")
        return None

def calculate_margin_forex(instrument_price, lot_size, chosen_lots, leverage, is_usdjpy=False):
    """
    Calculate margin for Forex pairs with special handling for pairs ending in JPY.
    """
    if is_usdjpy:
        # Margin in JPY converted to USD
        margin_jpy = (instrument_price * lot_size * chosen_lots) / leverage
        margin_usd = margin_jpy / instrument_price  # Convert to USD
        return margin_usd
    else:
        # Standard margin calculation
        return (instrument_price * lot_size * chosen_lots) / leverage

def calculate_margin_index(instrument_price, lot_size, chosen_lots, leverage):
    """
    Calculate margin for indices.
    """
    return (instrument_price * lot_size * chosen_lots) / leverage

def main():
    print("Welcome to the Margin Calculator!")
    print("Calculate the margin for Forex, commodities, indices, or cryptocurrencies.\n")

    # Get user input
    ticker = input("Enter the instrument ticker (e.g., XAUUSD, USDJPY, US30, BTCUSD): ").strip()
    leverage = float(input("Enter the leverage (e.g., 200, 50, 20): "))
    chosen_lots = float(input("Enter the number of lots you want to trade: "))

    # Map the ticker to Yahoo Finance format
    ticker = get_correct_ticker(ticker)

    # Determine the standard lot size for the instrument
    if ticker == "GC=F":  # Gold futures
        lot_size = 100
    elif ticker == "SI=F":  # Silver futures
        lot_size = 5000
    elif ticker in ["BTC-USD", "ETH-USD"]:  # Cryptocurrencies
        lot_size = 1
    elif ticker in ["^DJI", "^GSPC", "^NDX", "^FTSE", "^GDAXI", "^N225", "^HSI"]:  # Indices
        lot_size = 1
    else:
        lot_size = 100000  # Standard lot size for Forex pairs

    # Fetch the instrument price
    instrument_price = get_instrument_price(ticker)
    if instrument_price is None:
        print("Failed to fetch the price. Please check the ticker symbol and try again.")
        return

    # Check if it's a USDJPY pair (or ends with JPY)
    is_usdjpy = ticker.endswith("JPY=X")

    # Calculate margin
    if ticker in ["^DJI", "^GSPC", "^NDX", "^FTSE", "^GDAXI", "^N225", "^HSI"]:
        margin_required = calculate_margin_index(instrument_price, lot_size, chosen_lots, leverage)
    elif ticker in ["GC=F", "SI=F"]:
        margin_required = (instrument_price * lot_size * chosen_lots) / leverage
    else:
        margin_required = calculate_margin_forex(instrument_price, lot_size, chosen_lots, leverage, is_usdjpy)

    # Display the results
    print("\nCalculation Result:")
    print(f"Instrument: {ticker}")
    print(f"Price: ${instrument_price:,.2f}")
    print(f"Leverage: {leverage}x")
    print(f"Lot Size: {lot_size}")
    print(f"Number of Lots: {chosen_lots}")
    print(f"The required margin is: ${margin_required:.2f}\n")

if __name__ == "__main__":
    main()


Welcome to the Margin Calculator!
Calculate the margin for Forex, commodities, indices, or cryptocurrencies.

Enter the instrument ticker (e.g., XAUUSD, USDJPY, US30, BTCUSD): us30
Enter the leverage (e.g., 200, 50, 20): 200
Enter the number of lots you want to trade: 1

Calculation Result:
Instrument: ^DJI
Price: $42,992.21
Leverage: 200.0x
Lot Size: 1
Number of Lots: 1.0
The required margin is: $214.96

